In [1]:
import os 
import pandas as pd 
import scipy.stats as stats
import numpy as np

In [2]:
current_dir = os.getcwd()

# Original Dataset 
filename = '13 million Duolingo student learning traces.csv'
filepath = os.path.normpath(os.path.join(current_dir, '../data/raw/', filename))


chunk_size = 10000
chunks = []

for chunk in pd.read_csv(filepath, chunksize=chunk_size):
    chunk.drop_duplicates(inplace=True)
    chunk.dropna(inplace=True)
    chunks.append(chunk)

df = pd.concat(chunks, ignore_index=True)

In [3]:
# Check for na
nan_count = df.isna().sum().sum()
print(f'There are {nan_count} missing values in the dataset')

There are 0 missing values in the dataset


In [4]:
# Check for duplicates
duplicates_count = df.duplicated().sum().sum()
df.drop_duplicates(inplace=True)
print(f'There were {duplicates_count} duplicates in the dataset')

There were 0 duplicates in the dataset


In [5]:
# Outlier using z-score 
# The only outliers we should look at is history_seen, history_correct, delta 

threshold_z = 3
print(df['history_seen'][(np.abs(stats.zscore(df['history_seen'])) < threshold_z)].count())
print(df['history_correct'][(np.abs(stats.zscore(df['history_correct'])) < threshold_z)].count())
print(df['delta'][(np.abs(stats.zscore(df['delta'])) < threshold_z)].count())

12790181
12789624
12595913


In [6]:
# Show the values for 0.9 and 0.99 quantiles 
print(df['history_seen'].quantile(0.9), df['history_seen'].quantile(0.99))
print(round(df['delta'].quantile(0.9),2)/(60*60*60), round(df['delta'].quantile(0.99),2)/(60*60*60))

36.0 237.0
7.910547222222222 52.51042296296296


In [7]:
# Show the values on the edge of z-score  
print(df['history_seen'][(np.abs(stats.zscore(df['history_seen']))>=threshold_z)].min())
print((df['delta'][(np.abs(stats.zscore(df['delta']))>=threshold_z)].min()/(60*60*60)))

411
34.579351851851854


In [8]:
# Show rows higher than 0.9 qunatile 
# df[df['history_seen'] > 36.0] # 1 262 445 rows 

# Show rows higher than 0.99 quantile 
df[df['history_seen'] > 237].head() # 127 979 rows

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,session_seen,session_correct
187,1.0,1362082533,889,u:ezFh,es,en,4bdb859f599fa07dd5eecdab0acc2d34,a/a<pr>,261,229,1,1
188,1.0,1362082533,889,u:ezFh,es,en,41ba24ffb11fd09f5fc715d39cc6c1ea,qué/qué<prn><itg><m><sg>,243,217,1,1
200,1.0,1362082533,889,u:ezFh,es,en,ab724af62c7e36266ef3b66766d47489,te/prpers<prn><pro><p2><mf><sg>,260,220,1,1
206,1.0,1362082533,443,u:ezFh,es,en,c2c8a202a12411858f87d4dcef5b2998,no/no<adv>,463,434,1,1
212,1.0,1362082533,443,u:ezFh,es,en,99cd0848be9239250ffd99e11add7338,de/de<pr>,368,330,1,1


In [9]:
len(df['user_id'].unique()) # there are 115 222 users 

115222

In [10]:
df[df['history_seen'] > 237].groupby(df['user_id']).size().sort_values(ascending=False)   # There are only 1328 users with such score 
# The biggest user with such score has 12 322 rows 

user_id
u:bcH_    12322
u:cpBu     5561
u:goA      3695
u:NPs      3221
u:bkiW     3010
          ...  
u:ih1         1
u:hG1         1
u:dDx5        1
u:dG6         1
u:zz3         1
Length: 1328, dtype: int64

In [12]:
# Most of the users are "casual": they have low delta between words and low history_seen
# user u:bcH_ doesn't seem like a real user: seeing word have 913 times and only 583 correctly 

In [12]:
# Delete outliers


cols_to_trim = ['delta', 'history_seen', 'history_correct']

for col in cols_to_trim:
    upper_bound = df[col].quantile(0.99)
    df_after_removal = df[df[col] <= upper_bound]


print(f"Original rows: {len(df)}")
print(f"Trimmed rows:  {len(df_after_removal)}")
print(f"Rows removed:  {len(df) - len(df_after_removal)}")

# Z-score method
# df_outliers = df[['delta', 'history_seen', 'history_correct']]
# z = np.abs(stats.zscore(df_outliers))   


# df_after_removal = df[(z < threshold_z).all(axis=1)]


Original rows: 12854145
Trimmed rows:  12725745
Rows removed:  128400


In [13]:
df_after_removal

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,session_seen,session_correct
0,1.000000,1362076081,27649635,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,lernt/lernen<vblex><pri><p3><sg>,6,4,2,2
1,0.500000,1362076081,27649635,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,die/die<det><def><f><sg><nom>,4,4,2,1
2,1.000000,1362076081,27649635,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,mann/mann<n><m><sg><nom>,5,4,1,1
3,0.500000,1362076081,27649635,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,frau/frau<n><f><sg><nom>,6,5,2,1
4,1.000000,1362076081,27649635,u:FO,de,en,84920990d78044db53c1b012f5bf9ab5,das/das<det><def><nt><sg><nom>,4,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
12854140,0.800000,1363104897,368,u:i5D8,en,it,d5efc552aaea3109eb5388aa1ec8673d,the/the<det><def><sp>,6,4,5,4
12854141,0.800000,1363104897,368,u:i5D8,en,it,a826c47947d68549fa81e19cafa57ba0,eat/eat<vblex><pres>,4,4,5,4
12854142,1.000000,1363104897,368,u:i5D8,en,it,5e29d77697d23070a1fb92eb6c90e9b6,bread/bread<n><sg>,4,4,4,4
12854143,0.600000,1363104897,368,u:i5D8,en,it,cdfecc9247566d40bb964a218c54c783,drink/drink<vblex><pres>,3,2,5,3


In [14]:
df_after_removal['h_recall'] = df_after_removal['history_correct']/df_after_removal['history_seen']
df_after_removal.drop(columns=['lexeme_string'], inplace=True)

/var/folders/_b/l_819g5924d_y00npn9gt60h0000gn/T/ipykernel_51679/2704504019.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_after_removal['h_recall'] = df_after_removal['history_correct']/df_after_removal['history_seen']
/var/folders/_b/l_819g5924d_y00npn9gt60h0000gn/T/ipykernel_51679/2704504019.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_after_removal.drop(columns=['lexeme_string'], inplace=True)


In [17]:
print('max history before removal', df['history_seen'].max())

print('max history after removal', df_after_removal['history_seen'].max())

max history before removal 13518
max history after removal 498


In [18]:
df_after_removal['lang_combination'] = df_after_removal['ui_language'] + '-' + df_after_removal['learning_language']
df_after_removal.head()

/var/folders/_b/l_819g5924d_y00npn9gt60h0000gn/T/ipykernel_51679/823829646.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_after_removal['lang_combination'] = df_after_removal['ui_language'] + '-' + df_after_removal['learning_language']


,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,history_seen,history_correct,session_seen,session_correct,h_recall,lang_combination
0,1.0,1362076081,27649635,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,6,4,2,2,0.666667,en-de
1,0.5,1362076081,27649635,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,4,4,2,1,1.000000,en-de
2,1.0,1362076081,27649635,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,5,4,1,1,0.800000,en-de
3,0.5,1362076081,27649635,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,6,5,2,1,0.833333,en-de
4,1.0,1362076081,27649635,u:FO,de,en,84920990d78044db53c1b012f5bf9ab5,4,4,1,1,1.000000,en-de


In [19]:
filepath = os.path.normpath(os.path.join(current_dir, '../data/processed/'))
df_after_removal.to_csv(os.path.join(filepath, 'df_processed.csv'), sep=',', index=False, header=True)